In [1]:
import os

path = ("path")

In [2]:
addresses = []
# Getting my file w/ addresses
with open(path, 'r') as file:
    for line in file:
        if line.startswith("REWE Markt GmbH"):
            addresses.append(line.strip())


In [3]:
import re

# Preprocessing addresses
def is_valid_string(s):
    pattern = r'^[a-zA-Z0-9/.,!?"\'\s-]+$'
    return re.match(pattern, s) is not None

In [4]:
addresses = [address for address in addresses if is_valid_string(address) == True]

In [5]:
import pandas as pd

In [6]:
data = {'TEAM_MEMBER': [], 'STORE_ID': [], 'STORE_NAME': [], 'STORE_ADDRESS': [], 'STORE_NAME_ADDRESS': [], 'STORE_RATING': [], 'STORE_NUMBER_RATINGS': [], 'STORE_TYPE': [], 'STORE_PHONE': [], 'STORE_SERVICES': []}
store_df = pd.DataFrame(data)

In [7]:
store_df

,TEAM_MEMBER,STORE_ID,STORE_NAME,STORE_ADDRESS,STORE_NAME_ADDRESS,STORE_RATING,STORE_NUMBER_RATINGS,STORE_TYPE,STORE_PHONE,STORE_SERVICES


In [8]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import chromedriver_autoinstaller 
import undetected_chromedriver as uc 
from webdriver_manager.chrome import ChromeDriverManager
import cfscrape
import time



In [9]:
# Use cfscrape to bypass Cloudflare
scraper = cfscrape.create_scraper()
# Create a new undetected Chrome browser instance
options = uc.ChromeOptions()
options.headless = False
driver = uc.Chrome(options=options)

url = "https://www.rewe.de/angebote/berlin-friedenau/1350073/rewe-markt-rheinstr-30/"

response = scraper.get(url)
driver.get(url)

C:\Users\mjjyo\AppData\Local\Temp\ipykernel_6908\1465272153.py:5: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = False


In [10]:
store_name_list = []
store_address_list = []
store_phone_list = []
store_services_list = []
store_name_address_list = []

products_for_store_list = []

In [11]:
# Find and click the "Accept Cookies" button (if it exists)

accept_button = driver.find_element(By.XPATH, "//button[@id='uc-btn-accept-banner']")
ActionChains(driver).move_to_element(accept_button).click().perform()

time.sleep(5.23)

for i in addresses:

    driver.get(url)
    
    
    # Find and click the "change market" button
    change_market_button = driver.find_element(By.XPATH, "//button[contains(text(),'Markt ändern')]")
    ActionChains(driver).move_to_element(change_market_button).click().perform()

    # Wait for the popup to load
    time.sleep(2.34)

    # Find the address input field and enter the desired address
    address_input = driver.find_element(By.XPATH, "//input[@data-testid='gbmc-text-input']")
    ActionChains(driver).move_to_element(address_input).click().send_keys(i).perform()

    # Wait
    time.sleep(3.987)

    # Find and click the "Search" button
    search_button = driver.find_element(By.XPATH, "//button[contains(@class,'svelte-1rb3ttq') and contains(@class,'gbmc-underlined')]")
    ActionChains(driver).move_to_element(search_button).click().perform()

    # Wait for the page to load
    time.sleep(4.12)



    #Getting products

    product_list = []
    #article w/ class cor-offer-renderer-tile
    product_list = driver.find_elements(By.XPATH, "//article[contains(@class,'cor-offer-renderer-tile')]")

    final_product_list = []

    for item in product_list:
        try:
            image_element = item.find_element(By.XPATH, ".//img")
            image_url = image_element.get_attribute('src')
            final_product_list.append((image_url, item.text))
        except:
            pass

    # Find and click the "store info" button
    change_store_link = driver.find_element(By.XPATH, "//a[contains(text(),'Alles zum Markt')]")
    ActionChains(driver).move_to_element(change_store_link).click().perform()
    
    # Find store name and phone
    try: 
        store_name = driver.find_element(By.XPATH, "//h1[contains(@class,'market-name')]")
        store_name = store_name.text
    except:
        pass
        
    try:
        store_phone = driver.find_element(By.XPATH, "//div[contains(@class,'market-contact')]")
        store_phone = store_phone.text
    except:
        pass

    # Find services
    market_services_div = None
    try:
        market_services_div = driver.find_element(By.XPATH, "//div[contains(@class,'market-services')]")
    except:
        pass
        
    # If the div was found get contents
    if market_services_div:
        store_services = market_services_div.text
        store_services_list.append(f"In-store shopping, {store_services}")
    else:
        store_services_list.append('In-store shopping')

        
    go_back_to_page = driver.find_element(By.XPATH, "//a[contains(text(),'Angebote')]")
    ActionChains(driver).move_to_element(go_back_to_page).click().perform()

     
    store_name_list.append(store_name)
    store_phone_list.append(store_phone)
    store_address_list.append(i)
    products_for_store_list.append(final_product_list)
    
    driver.switch_to.window(driver.window_handles[0])


In [12]:
driver.close()


In [13]:
# Adjusting the received string to be better on CSV
final_store_phone_list = []
for number in store_phone_list:
    for item in number.split('Telefon: '):
        for subitem in item.split('\nREWE Markt GmbH'):
            try:
                (subitem.split()[0])
                final_store_phone_list.append(subitem)
            except IndexError:
                continue

In [14]:
# Cleaning store address list
store_address_list = list(store_df['STORE_ADDRESS'])

In [15]:
store_address_list = [item.split(',')[1::] for item in store_address_list]

In [16]:
store_address_list = [' '.join(item).strip() for item in store_address_list]

In [ ]:
# Adding to csv

In [17]:
store_df['STORE_NAME'] = store_name_list

In [18]:
store_df['STORE_ADDRESS'] = store_address_list

In [19]:
store_df['STORE_PHONE'] = final_store_phone_list

In [20]:
store_df['STORE_SERVICES'] = store_services_list

In [21]:
store_df['STORE_NAME_ADDRESS'] = store_df.apply(lambda row: row['STORE_NAME'] + ' ' + row['STORE_ADDRESS'], axis=1)


In [22]:
store_df['TEAM_MEMBER'] = 'Michael Johnson'

In [23]:
store_df

,TEAM_MEMBER,STORE_ID,STORE_NAME,STORE_ADDRESS,STORE_NAME_ADDRESS,STORE_RATING,STORE_NUMBER_RATINGS,STORE_TYPE,STORE_PHONE,STORE_SERVICES
0,Michael Johnson,NaN,REWE Supermarkt,NaN,NaN,NaN,NaN,NaN,030-85074370,In-store shopping
1,Michael Johnson,NaN,REWE Supermarkt,NaN,NaN,NaN,NaN,NaN,030-83202917,"In-store shopping, Abholservice"
2,Michael Johnson,NaN,REWE Supermarkt,NaN,NaN,NaN,NaN,NaN,030-30207371,"In-store shopping, Abholservice"
3,Michael Johnson,NaN,REWE Supermarkt,NaN,NaN,NaN,NaN,NaN,030-30207371,In-store shopping
4,Michael Johnson,NaN,REWE Supermarkt,NaN,NaN,NaN,NaN,NaN,030-53029550,In-store shopping
...,...,...,...,...,...,...,...,...,...,...
63,Michael Johnson,NaN,REWE Supermarkt,NaN,NaN,NaN,NaN,NaN,030-44677270,In-store shopping
64,Michael Johnson,NaN,REWE Supermarkt,NaN,NaN,NaN,NaN,NaN,030-80406790,In-store shopping
65,Michael Johnson,NaN,REWE Supermarkt,NaN,NaN,NaN,NaN,NaN,030-40043237,In-store shopping
66,Michael Johnson,NaN,REWE Supermarkt,NaN,NaN,NaN,NaN,NaN,030-40393105,In-store shopping


In [25]:
image_list = []
information_list = []
deal_list = []
price_list = []
name_list = []

In [2]:
# Getting details on products and putting in csv


# For each store
for num in range(len(products_for_store_list)):
    # For product in store
    for i in products_for_store_list[num]:
        
        image_list.append(i[0])
        food_text = i[1]
        
        # Get rid of these for easier parsing

        if 'Knaller' in food_text:
            details = food_text.split('Knaller')
    #         details = ' '.join(details)
        elif 'Aktion' in food_text:
            details = food_text.split('Aktion')
    #         details = ' '.join(details)

        # Finding deal
    
        information = details[0]
        deal = 'None'
        if 'noch günstiger mit App Coupon' in information:
            information = information.split('noch günstiger mit App Coupon')
            information = ' '.join(information[1::])
            deal = 'Even cheaper with App Coupon'
        if 'Sensationspreis' in information:
            information = information.split('Sensationspreis')
            information = ' '.join(information[1::])

        # Finding info
        
        information = information.split('\n')
        information = list(filter(lambda x: x != '', information))

        name = information[0]
        item_details = information[1]
        price = details[1].split('\n')[1]


        information_list.append(item_details)
        deal_list.append(deal)
        price_list.append(price)
        name_list.append(name)

In [27]:


image_list = image_list[:11747]


In [28]:
product_data = {'Store': [], 'Image': [], 'Name': [], 'Item Details': [], 'Price': [], 'Deal': []}

product_csv = pd.DataFrame(product_data)

In [29]:
product_csv['Image'] = image_list

In [30]:
product_csv['Store'] = 'REWE Supermarkt'

In [31]:
product_csv['Name'] = name_list

In [32]:
product_csv['Price'] = price_list

In [33]:
product_csv['Deal'] = deal_list

In [34]:
product_csv['Item Details'] = information_list

In [36]:
product_csv.to_csv('path', index=False)